# Автоматическое управление временными пользователями

#### Скрипт для остановки сессии и удаления пользователя по заданным тайм аутам (время неактивной работы пользователя, время активной работы сеанса). Останавливает сессии и удаляет пользователей с обычными правами (не администраторов)

#### <font color="Brown">!!! Перед запуском основных функций, запустить функции обертки и консольные вызовы</font>

### Текущие значения

In [3]:
time_out_user, time_out_sessions = get_time_outs()
print(f"Время неактивной работы пользователя {time_out_user}")
print(f"Время длительности сеанса {time_out_sessions}")

Время неактивной работы пользователя 100
Время длительности сеанса 100


### Установка значений

In [ ]:
new_time_out_user = int(input('Введите время неактивной работы пользователя, после которого он будет удален (сек):'))
new_time_out_sessions = int(input('Введите время длительности сеанса, после которого сеанс принудительно завершится (сек):'))

config = {"time_out_user": new_time_out_user, "time_out_sessions": new_time_out_sessions}
save_config(config)

Введите время неактивной работы пользователя, после которого он будет удален (сек): 600


### Перезапуск скрипта

In [8]:
restart()

### Запуск скрипта

In [9]:
start()

### Остановка скрипта

In [5]:
stop()

### Посмотреть статус работы скрипта

In [10]:
print_status()

● delete-users.service - Delete users
     Loaded: loaded (/etc/systemd/system/delete-users.service; enabled; preset: enabled)
     Active: active (running) since Fri 2023-12-22 21:26:19 +04; 4min 16s ago
   Main PID: 3183 (python3)
      Tasks: 1 (limit: 8685)
     Memory: 15.2M
        CPU: 130ms
     CGroup: /system.slice/delete-users.service
             └─3183 /usr/bin/python3 /home/godji/tljh/delete_users.py

дек 22 21:26:19 vivobook systemd[1]: Started delete-users.service - Delete users.
-----------------
● delete-sessions.service - Delete sessions
     Loaded: loaded (/etc/systemd/system/delete-sessions.service; enabled; preset: enabled)
     Active: active (running) since Fri 2023-12-22 21:26:20 +04; 4min 15s ago
   Main PID: 3219 (python3)
      Tasks: 1 (limit: 8685)
     Memory: 16.6M
        CPU: 978ms
     CGroup: /system.slice/delete-sessions.service
             └─3219 /usr/bin/python3 /home/godji/tljh/delete_sessions.py

дек 22 21:26:20 vivobook systemd[1]: Started de

### Функции обертки

In [1]:
import json
import os
file_path = '/opt/tljh/config.json'

def restart():
    daemon_reload()
    # user
    stop_delete_users()
    enable_delete_users()
    start_delete_users()
    #session
    stop_delete_sessions()
    enable_delete_sessions()
    start_delete_sessions()
    
def start():
    start_delete_users()
    start_delete_sessions()
    
def stop():
    stop_delete_users()
    stop_delete_sessions()

def print_status():
    s = status_delete_users()
    print('\n'.join(s))
    print('-----------------')
    s = status_delete_sessions()
    print('\n'.join(s))

def get_time_outs():
    config = load_config()
    if config is None:
        # Создаем новый конфиг, если файл не найден
        config = {"time_out_user": 600, "time_out_sessions": 600}
        save_config(config)
    time_out_user = config["time_out_user"]
    time_out_sessions = config["time_out_sessions"]
    return time_out_user, time_out_sessions

def load_config():
    try:
        file_descriptor = os.open(file_path, os.O_RDONLY)
        with os.fdopen(file_descriptor, 'r') as file:
            config = json.load(file)
        return config
    except FileNotFoundError:
        print(f"Файл конфигурации {file_path} не найден.")
        return None

def save_config(config):
    file_descriptor = os.open(file_path, os.O_WRONLY | os.O_CREAT | os.O_TRUNC, 0o666)
    with os.fdopen(file_descriptor, 'w') as file:
        json.dump(config, file, indent=2)

### Консольные вызовы

In [2]:
def daemon_reload():
    !sudo systemctl daemon-reload
    
def enable_delete_users():
    !sudo systemctl enable delete-users.service

def start_delete_users():
    !sudo systemctl start delete-users.service

def stop_delete_users():
    !sudo systemctl stop delete-users.service

def status_delete_users():
    s = !sudo systemctl status delete-users.service
    return s


def enable_delete_sessions():
    !sudo systemctl enable delete-sessions.service

def start_delete_sessions():
    !sudo systemctl start delete-sessions.service

def stop_delete_sessions():
    !sudo systemctl stop delete-sessions.service

def status_delete_sessions():
    s = !sudo systemctl status delete-sessions.service
    return s

def get_file():
    !sudo ln /opt/history_sessions.log /opt/history_sessions.log